# Quantitative Value Strategy

"Value investing" is investing the stocks that are cheapest relative to common measures of business value (e.g. earnings or assets).

In this project, we are going to build an investing strategy that selects the 50 stocks with the best value metrics and calculate recommended trades for an equal-weight portfolio of these 50 stocks.

In [2]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from scipy import stats

In [3]:
stocks_full = pd.read_csv('sp_500_stocks.csv')
stocks = pd.DataFrame({'ticker': stocks_full['Symbol']})
from secrets import IEX_CLOUD_API_TOKEN

## Making & Parsing our first API call

We will build a simple value screener that ranks securities based on a single metric: the price-to-earnings ratio.

In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2327687132791,
 'week52high': 138.76,
 'week52low': 56.04,
 'week52change': 0.806114158284684,
 'sharesOutstanding': 17690748701,
 'float': 0,
 'avg10Volume': 97758953,
 'avg30Volume': 105334575,
 'day200MovingAvg': 115.1,
 'day50MovingAvg': 128.11,
 'employees': 0,
 'ttmEPS': 3.43,
 'ttmDividendRate': 0.8151711359665899,
 'dividendYield': 0.006173851396442389,
 'nextDividendDate': '0',
 'exDividendDate': '2020-11-01',
 'nextEarningsDate': '0',
 'peRatio': 39.6469189490834,
 'beta': 1.1943699274973394,
 'maxChangePercent': 50.65763338660246,
 'year5ChangePercent': 4.629185256972964,
 'year2ChangePercent': 2.5001067346534462,
 'year1ChangePercent': 0.8273754770441853,
 'ytdChangePercent': 0,
 'month6ChangePercent': 0.47095140420070974,
 'month3ChangePercent': 0.14941791435030774,
 'month1ChangePercent': 0.119985227318389,
 'day30ChangePercent': 0.08348580419120956,
 'day5ChangePercent': 0.00557106027599267}

In [5]:
pe_ratio = data['peRatio']
pe_ratio

39.6469189490834

## Execute a batch API call & Build our DataFrame

In [6]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst"""
    for i in range(0, len(lst), n):
        yield lst[i:(i+n)]


symbol_groups = list(chunks(stocks['ticker'], 100))
symbol_strings = []

for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

    
colns = ['Ticker', 'Stock Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']
final_df = pd.DataFrame(columns = colns)

In [7]:
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_df = final_df.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['peRatio'],
                    'N/A',
                ],
                index = colns,
            ),
            ignore_index = True
        )

final_df

,Ticker,Stock Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,MMM,177.06,20.69,N/A
1,AOS,55.82,29.36,N/A
2,ABT,111.88,59.34,N/A
3,ABBV,108.21,24.2,N/A
4,ABMD,330.30,71.2,N/A
5,ACN,261.71,34.34,N/A
6,ATVI,93.40,33.49,N/A
7,ADBE,504.63,63.66,N/A
8,AAP,159.16,23.08,N/A
9,AMD,92.24,131.9,N/A


## Remove glamour stocks 

"glamour stocks" are the opposite of "value stocks".

We will sort the dataframe by stocks' price-to-earnings ratio and drop all stocks outside the top 50

In [8]:
final_df.sort_values('Price-to-Earnings Ratio', inplace=True)
final_df = final_df[final_df['Price-to-Earnings Ratio'] > 0]
final_df = final_df[:50]

final_df.reset_index(inplace=True)
final_df.drop('index', axis=1, inplace=True)

final_df

,Ticker,Stock Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,39.15,2.27,N/A
1,NLOK,21.16,4.19,N/A
2,UNM,23.64,5.04,N/A
3,BIO,583.61,5.13,N/A
4,AIV,5.43,5.34,N/A
5,KIM,15.16,6.78,N/A
6,ALL,110.80,7.7,N/A
7,MET,47.01,8,N/A
8,EBAY,50.74,8.13,N/A
9,BIIB,245.65,8.17,N/A


## Calculate number of shares to buy

refer to project2 about momentum investment

In [9]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio: ')
    
    try:
        float(portfolio_size)
    except ValueError:
        print('That is not a number! \nPlease try again.')
        portfolio_size = input('Enter the size of your portfolio: ')

In [10]:
portfolio_input()

Enter the size of your portfolio: 1000000


In [11]:
position_size = float(portfolio_size)/len(final_df)

for i in range(0, len(final_df)):
    final_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_df.loc[i, 'Stock Price'])

final_df

,Ticker,Stock Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,39.15,2.27,510
1,NLOK,21.16,4.19,945
2,UNM,23.64,5.04,846
3,BIO,583.61,5.13,34
4,AIV,5.43,5.34,3683
5,KIM,15.16,6.78,1319
6,ALL,110.80,7.7,180
7,MET,47.01,8,425
8,EBAY,50.74,8.13,394
9,BIIB,245.65,8.17,81


## Build a better value strategy

The flaw of the price-to-earnings ratio strategy does not work well with stocks with negative earnings. 

Similarly, the stocks that buyback their own shares are difficult to value using the price-to-book ratio.

We typically use a composite basket of valuation metrics to build robust quantitative value strategies (RV). 

Here, we will filter for stocks with the lowest percentiles on the following metrics:
* price-to-earnings ratio
* price-to-book ratio
* price-to-sales ratio
* enterprise value divided by earnings before interest, taxes, depreciation, and amortization (EV/EBITDA)
* enterprise value divided by gross profit

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data.

In [12]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=advanced-stats,quote&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

pe_ratio = data[symbol]['quote']['peRatio']

pb_ratio = data[symbol]['advanced-stats']['priceToBook']

ps_ratio = data[symbol]['advanced-stats']['priceToSales']

enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

In [13]:
rv_colns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'P/E Ratio',
    'PE Percentile',
    'P/B Ratio',
    'PB Percentile',
    'P/S Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_df = pd.DataFrame(columns = rv_colns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=advanced-stats,quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
            
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
        
        rv_df = rv_df.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    'N/A',
                    data[symbol]['quote']['peRatio'],
                    'N/A',
                    data[symbol]['advanced-stats']['priceToBook'],
                    'N/A',
                    data[symbol]['advanced-stats']['priceToSales'],
                    'N/A',
                    ev_to_ebitda,
                    'N/A',
                    ev_to_gross_profit,
                    'N/A',
                    'N/A'
                ],
                index = rv_colns,
            ),
            ignore_index = True
        )

rv_df


,Ticker,Price,Number of Shares to Buy,P/E Ratio,PE Percentile,P/B Ratio,PB Percentile,P/S Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,MMM,182.63,N/A,21.42,N/A,8.83,N/A,3.22,N/A,14.461970,N/A,7.575480,N/A,N/A
1,AOS,56.43,N/A,28.53,N/A,5.11,N/A,3.25,N/A,17.205845,N/A,7.733929,N/A,N/A
2,ABT,113.63,N/A,57.65,N/A,6.3,N/A,6.2,N/A,32.414147,N/A,11.470960,N/A,N/A
3,ABBV,108.55,N/A,23.5,N/A,12.97,N/A,4.75,N/A,15.896287,N/A,9.756097,N/A,N/A
4,ABMD,337.10,N/A,74.6,N/A,12.63,N/A,19.01,N/A,58.453903,N/A,21.589896,N/A,N/A
5,ACN,269.59,N/A,33.58,N/A,10.19,N/A,4,N/A,20.229682,N/A,11.977768,N/A,N/A
6,ATVI,96.58,N/A,33.68,N/A,5.14,N/A,9.62,N/A,21.506810,N/A,12.301236,N/A,N/A
7,ADBE,513.75,N/A,64.27,N/A,21.39,N/A,19.47,N/A,57.793947,N/A,21.714463,N/A,N/A
8,AAP,163.91,N/A,23.32,N/A,2.88,N/A,1.13,N/A,10.627278,N/A,2.474772,N/A,N/A
9,AMD,95.85,N/A,129.55,N/A,29.56,N/A,12.93,N/A,75.919375,N/A,28.892597,N/A,N/A


## Dealing with missing data

We will use pandas' isnull method to identify missing data

Two approaches to consider:
* drop missing data from the dataset (`dropna` method)
* replace missing data with a new value (`fillna` method)

Here, we will replace missing data with the average non-NaN data from that column

In [14]:
rv_df[rv_df.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,P/E Ratio,PE Percentile,P/B Ratio,PB Percentile,P/S Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
24,GOOG,1811.19,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
43,AON,212.71,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
65,BRK.B,237.51,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
95,CTL,11.00,N/A,9.97,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
145,DISCK,26.21,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
158,ETFC,50.30,N/A,14.71,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
175,EVRG,56.37,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
189,FRC,151.43,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
201,FOX,29.15,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
325,MYL,16.46,N/A,31.72,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


In [15]:
for col in ['P/E Ratio', 'P/B Ratio', 'P/S Ratio', 'EV/EBITDA', 'EV/GP']:
    rv_df[col].fillna(rv_df[col].mean(), inplace=True)

In [16]:
rv_df[rv_df.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,P/E Ratio,PE Percentile,P/B Ratio,PB Percentile,P/S Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculate value percentiles

In [17]:
metrics = {
    'P/E Ratio': 'PE Percentile',
    'P/B Ratio': 'PB Percentile',
    'P/S Ratio': 'PS Percentile',
    'EV/EBITDA': 'EV/EBITDA Percentile',
    'EV/GP': 'EV/GP Percentile',
}

for row in rv_df.index:
    for metric in metrics.keys():
        rv_df.loc[row, metrics[metric]] = stats.percentileofscore(rv_df[metric], rv_df.loc[row, metric])/100 # divide 100 for presentation as %
        
for metric in metrics.values():
    print(rv_df[metric])
    
rv_df

0       0.415842
1       0.546535
2       0.877228
3       0.450495
4       0.912871
5       0.658416
6       0.665347
7        0.89703
8       0.447525
9       0.966337
10     0.0554455
11      0.207921
12      0.855446
13      0.625743
14           0.6
15      0.954455
16      0.756436
17      0.762376
18      0.746535
19      0.469307
20      0.708911
21      0.376238
22      0.165347
23      0.687129
24       0.79703
25      0.952475
26      0.948515
27      0.479208
28      0.455446
29      0.140594
         ...    
475     0.760396
476     0.968317
477     0.669307
478     0.720792
479     0.681188
480     0.815842
481     0.413861
482     0.661386
483     0.621782
484     0.463366
485     0.918812
486     0.505941
487     0.906931
488    0.0118812
489     0.282178
490    0.0831683
491     0.839604
492     0.241584
493     0.964356
494     0.534653
495     0.110891
496     0.482178
497      0.79703
498     0.879208
499     0.914851
500     0.638614
501     0.776238
502     0.0198

,Ticker,Price,Number of Shares to Buy,P/E Ratio,PE Percentile,P/B Ratio,PB Percentile,P/S Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,MMM,182.63,N/A,21.420000,0.415842,8.830000,0.766337,3.22000,0.515842,14.461970,0.425743,7.575480,0.483168,N/A
1,AOS,56.43,N/A,28.530000,0.546535,5.110000,0.59703,3.25000,0.519802,17.205845,0.526733,7.733929,0.49703,N/A
2,ABT,113.63,N/A,57.650000,0.877228,6.300000,0.668317,6.20000,0.768317,32.414147,0.839604,11.470960,0.728713,N/A
3,ABBV,108.55,N/A,23.500000,0.450495,12.970000,0.855446,4.75000,0.677228,15.896287,0.481188,9.756097,0.655446,N/A
4,ABMD,337.10,N/A,74.600000,0.912871,12.630000,0.847525,19.01000,0.976238,58.453903,0.914851,21.589896,0.946535,N/A
5,ACN,269.59,N/A,33.580000,0.658416,10.190000,0.815842,4.00000,0.591089,20.229682,0.629703,11.977768,0.750495,N/A
6,ATVI,96.58,N/A,33.680000,0.665347,5.140000,0.60198,9.62000,0.875248,21.506810,0.673267,12.301236,0.762376,N/A
7,ADBE,513.75,N/A,64.270000,0.89703,21.390000,0.918812,19.47000,0.978218,57.793947,0.912871,21.714463,0.948515,N/A
8,AAP,163.91,N/A,23.320000,0.447525,2.880000,0.40297,1.13000,0.166337,10.627278,0.243564,2.474772,0.128713,N/A
9,AMD,95.85,N/A,129.550000,0.966337,29.560000,0.956436,12.93000,0.938614,75.919375,0.976238,28.892597,0.980198,N/A


## Calculate RV score

The RV score is the value score that we will use to filter for stocks in this investing strategy.

RV score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

In [18]:
from statistics import mean

for row in rv_df.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_df.loc[row, metrics[metric]])
    rv_df.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_df

,Ticker,Price,Number of Shares to Buy,P/E Ratio,PE Percentile,P/B Ratio,PB Percentile,P/S Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,MMM,182.63,N/A,21.420000,0.415842,8.830000,0.766337,3.22000,0.515842,14.461970,0.425743,7.575480,0.483168,0.521386
1,AOS,56.43,N/A,28.530000,0.546535,5.110000,0.59703,3.25000,0.519802,17.205845,0.526733,7.733929,0.49703,0.537426
2,ABT,113.63,N/A,57.650000,0.877228,6.300000,0.668317,6.20000,0.768317,32.414147,0.839604,11.470960,0.728713,0.776436
3,ABBV,108.55,N/A,23.500000,0.450495,12.970000,0.855446,4.75000,0.677228,15.896287,0.481188,9.756097,0.655446,0.62396
4,ABMD,337.10,N/A,74.600000,0.912871,12.630000,0.847525,19.01000,0.976238,58.453903,0.914851,21.589896,0.946535,0.919604
5,ACN,269.59,N/A,33.580000,0.658416,10.190000,0.815842,4.00000,0.591089,20.229682,0.629703,11.977768,0.750495,0.689109
6,ATVI,96.58,N/A,33.680000,0.665347,5.140000,0.60198,9.62000,0.875248,21.506810,0.673267,12.301236,0.762376,0.715644
7,ADBE,513.75,N/A,64.270000,0.89703,21.390000,0.918812,19.47000,0.978218,57.793947,0.912871,21.714463,0.948515,0.931089
8,AAP,163.91,N/A,23.320000,0.447525,2.880000,0.40297,1.13000,0.166337,10.627278,0.243564,2.474772,0.128713,0.277822
9,AMD,95.85,N/A,129.550000,0.966337,29.560000,0.956436,12.93000,0.938614,75.919375,0.976238,28.892597,0.980198,0.963564


In [19]:
rv_df.sort_values(by='RV Score', inplace=True)
rv_df = rv_df[:50]
rv_df.reset_index(drop=True, inplace=True)

In [20]:
rv_df

,Ticker,Price,Number of Shares to Buy,P/E Ratio,PE Percentile,P/B Ratio,PB Percentile,P/S Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,23.72,N/A,4.92,0.154455,0.4431,0.049505,0.4021,0.0316832,2.515216,0.0316832,0.384112,0.0039604,0.0542574
1,AIG,37.92,N/A,-6.77,0.106931,0.5163,0.0574257,0.7369,0.0633663,4.303032,0.039604,0.688899,0.0138614,0.0562376
2,AAL,15.85,N/A,-1.17,0.140594,-1.8100,0.0455446,0.3926,0.0277228,-5.941849,0.0277228,1.355955,0.0514851,0.0586139
3,MET,49.02,N/A,7.80,0.162376,0.5986,0.0613861,0.6720,0.0574257,4.442323,0.0415842,0.628264,0.0118812,0.0669307
4,HFC,25.96,N/A,-27.20,0.0673267,0.8037,0.0792079,0.2963,0.0138614,6.211729,0.0653465,2.163233,0.110891,0.0673267
5,F,9.18,N/A,-16.69,0.0811881,1.1100,0.122772,0.2745,0.0118812,4.603221,0.0435644,1.998872,0.0970297,0.0712871
6,HIG,51.27,N/A,10.26,0.190099,1.0000,0.10198,0.8883,0.0930693,4.156035,0.0376238,0.860732,0.019802,0.0885149
7,ALL,113.66,N/A,8.00,0.165347,1.3300,0.167327,0.7576,0.0673267,2.532358,0.0336634,0.745889,0.0158416,0.089901
8,BA,215.82,N/A,-27.26,0.0653465,-10.3000,0.0316832,2.0900,0.352475,-28.570998,0.0158416,-200.082035,0.0019802,0.0934653
9,LNC,51.26,N/A,13.12,0.228713,0.4730,0.0514851,0.5643,0.0435644,9.292810,0.178218,0.556473,0.00990099,0.102376


## Calculate the number of shares to buy

In [ ]:
position_size = float(portfolio_size) / len(rv_df.index)

for i in range(0, len(rv_df['Ticker'])-1):
    rv_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_df['Price'][i])
    
rv_df

/Users/bingjieouyang/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


## Format Excel output

In [ ]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine = 'xlsxwriter')
rv_df.to_excel(writer, sheet_name = 'value', index=False)

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

percent_format = writer.book.add_format(
    {
        'num_format': '0.0%',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

float_format = writer.book.add_format(
    {
        'num_format': '0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [ ]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Price', dollar_format],
    'C': ['Number of Shares to Buy', integer_format],
    'D': ['P/E Ratio', percent_format],
    'E': ['PE Percentile', percent_format],
    'F': ['P/B Ratio', percent_format],
    'G': ['PB Percentile', percent_format],
    'H': ['P/S Ratio', percent_format],
    'I': ['PS Percentile', percent_format],
    'J': ['EV/EBITDA', percent_format],
    'K': ['EV/EBITDA Percentile', percent_format],
    'L': ['EV/GP', integer_format],
    'M': ['EV/GP Percentile', percent_format],
    'N': ['RV Score', percent_format],
}

for column in column_formats.keys():
    writer.sheets['value'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['value'].write(f'{column}1', column_formats[column][0], string_format)

writer.save()